In [1]:
sc.install_pypi_package("numpy")
sc.install_pypi_package("pandas")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1742178247847_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1742178247847_0001/container_1742178247847_0001_01_000001/tmp/spark-9ec5c0b1-1ab9-4ace-b1d4-b01bb09de609
    Can't uninstall 'python-dateutil'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.17.18 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.

In [2]:
import pyspark 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RandomForest").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

You are working with 1 core(s)

In [7]:
spark = SparkSession.builder \
    .appName("RandomForest") \
    .config("spark.executor.cores", "4") \
    .config("spark.cores.max", "4") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

You are working with 1 core(s)

In [9]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df = spark.read.parquet('s3://raw-data-bitcoin/bitcoin_cleaned_data/20250316_033643_00015_zb83s_8e7bb85c-14bb-4651-90fb-6c419f922ba1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
print(df.head())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(date=datetime.datetime(2021, 1, 7, 17, 49), open=39561.22, high=39561.23, low=39525.45, close=39551.56, volume=32.372589)

In [13]:
print(df.dtypes)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('date', 'timestamp'), ('open', 'double'), ('high', 'double'), ('low', 'double'), ('close', 'double'), ('volume', 'double')]

In [14]:
print(df.describe())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, open: string, high: string, low: string, close: string, volume: string]

In [16]:
from pyspark.sql.functions import col, to_timestamp, year, month, dayofmonth

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df = df.withColumn("date", to_timestamp(col("date")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Extract year, month, and day from the date if you want to use them as features
df = df.withColumn("year", year(col("date"))) \
       .withColumn("month", month(col("date"))) \
       .withColumn("day", dayofmonth(col("date")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_cleaned = df.dropna(subset=["open", "high", "low", "close", "volume"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Ensure that all features are numeric (convert string columns to floats if necessary)
df_cleaned = df_cleaned.withColumn("open", col("open").cast("float")) \
                       .withColumn("high", col("high").cast("float")) \
                       .withColumn("low", col("low").cast("float")) \
                       .withColumn("close", col("close").cast("float")) \
                       .withColumn("volume", col("volume").cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Select features (drop the target column 'close' from the features)
feature_columns = ["open", "high", "low", "volume", "year", "month", "day"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Assemble features into a single vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_ready = assembler.transform(df_cleaned)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Split data into training and testing datasets (80% train, 20% test)
train_data, test_data = df_ready.randomSplit([0.8, 0.2], seed=1234)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Initialize RandomForestRegressor (default parameters for quick execution)
rf = RandomForestRegressor(featuresCol="features", labelCol="close", numTrees=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Train the model
model = rf.fit(train_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Make predictions on the test data
predictions = model.transform(test_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…